In [2]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load the two YOLO models
goalpost_model = YOLO('../best.pt')  # Replace with your goalpost model path
ball_model = YOLO('../ball.pt')         # Replace with your ball model path



In [3]:
def is_goal(ball_bbox, goalpost_bbox):
    """
    Determine if the ball has crossed the goal line.

    Args:
        ball_bbox (tuple): Bounding box of the ball (x_min, y_min, x_max, y_max).
        goalpost_bbox (tuple): Bounding box of the goalpost (x_min, y_min, x_max, y_max).
    
    Returns:
        bool: True if a goal is detected, False otherwise.
    """
    # Ball bounding box
    x_ball_min, y_ball_min, x_ball_max, y_ball_max = ball_bbox
    x_ball_center = (x_ball_min + x_ball_max) / 2
    y_ball_center = (y_ball_min + y_ball_max) / 2

    # Goalpost bounding box
    x_goal_min, y_goal_min, x_goal_max, y_goal_max = goalpost_bbox

    # Check if ball crosses the goal line
    goal_line_x = x_goal_max  # Assuming goal line is at x_goal_max
    if x_ball_center > goal_line_x:
        # Check if ball is within the vertical range of the goalpost
        if y_ball_center >= y_goal_min and y_ball_center <= y_goal_max:
            return True  # Goal detected

    return False

# Process video frame-by-frame


In [9]:
def process_video(video_path, output_path, max_frame_num):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Output video writer
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_num = 0
    while cap.isOpened() and frame_num < max_frame_num:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect objects using goalpost model and filter for 'goalPost' class
        goalpost_results = goalpost_model(frame)
        goalpost_bboxes = [
            (int(box[0]), int(box[1]), int(box[2]), int(box[3]))
            for box, cls in zip(goalpost_results[0].boxes.xyxy.cpu().numpy(), goalpost_results[0].boxes.cls.cpu().numpy())
            if goalpost_results[0].names[int(cls)] == "goalPost"
        ]

        # Detect objects using ball model
        ball_results = ball_model(frame)
        ball_bboxes = [
            (int(box[0]), int(box[1]), int(box[2]), int(box[3]))
            for box in ball_results[0].boxes.xyxy.cpu().numpy()
        ]

        goal_detected = False

        # Check for goal
        for ball_bbox in ball_bboxes:
            for goalpost_bbox in goalpost_bboxes:
                if is_goal(ball_bbox, goalpost_bbox):
                    goal_detected = True
                    break
            if goal_detected:
                break

        # Annotate frame
        for (x_min, y_min, x_max, y_max) in goalpost_bboxes:
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green for goalpost
            cv2.putText(frame, "Goalpost", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        for (x_min, y_min, x_max, y_max) in ball_bboxes:
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)  # Red for ball
            cv2.putText(frame, "Ball", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        # Add goal text
        if goal_detected:
            cv2.putText(frame, "GOAL!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        else:
            cv2.putText(frame, "NO GOAL", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        # Write frame to output
        out.write(frame)
        frame_num += 1

    cap.release()
    out.release()
    print("Processing complete. Output saved to:", output_path)

# Run the video processing with a frame limit


In [10]:
process_video('../video_input.mp4', './goal_output.mp4', max_frame_num=3552)

0: 384x640 1 goalPost, 1 goalie, 1 player, 37.4ms
Speed: 4.3ms preprocess, 37.4ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.1ms
Speed: 4.6ms preprocess, 37.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalPost, 1 goalie, 1 player, 37.4ms
Speed: 3.7ms preprocess, 37.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 32.5ms
Speed: 4.0ms preprocess, 32.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalPost, 1 goalie, 32.4ms
Speed: 4.4ms preprocess, 32.4ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 33.0ms
Speed: 2.2ms preprocess, 33.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalPost, 1 goalie, 32.4ms
Speed: 4.3ms preprocess, 32.4ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2